In [19]:
import os, warnings
import matplotlib.pyplot as plt
from matplotlib import gridspec
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)
plt.rc('image', cmap='magma')
warnings.filterwarnings("ignore") 


import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory


Kaggle Tutorial for convolutional image classifiers: 1 base + 1 head

- base: extracts features from images
- head: classifies image based on extracted features 

build a classifier with transfer learning on pretrained base

In [12]:
# Data Pipeline
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE


In [9]:
path = os.getcwd()

In [10]:
ds_train_ = image_dataset_from_directory(
    path+'/input/car-or-truck/train',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=True,
)
ds_valid_ = image_dataset_from_directory(
    path+'/input/car-or-truck/valid',
    labels='inferred',
    label_mode='binary',
    image_size=[128, 128],
    interpolation='nearest',
    batch_size=64,
    shuffle=False,
)

Found 5117 files belonging to 2 classes.
Found 5051 files belonging to 2 classes.


In [13]:
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

Use VGG16

In [18]:
from keras.applications.vgg16 import VGG16


553467096/553467096 [==============================] - 173s 0us/step


In [24]:
model = VGG16(input_shape=(128,128,3),weights=None)

In [25]:
seq_model = keras.Sequential([
    model,
    layers.Flatten(),
    layers.Dense(6, activation='relu'),
    layers.Dense(1, activation = 'sigmoid'),
])


In [26]:
seq_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy']
)

In [27]:
history = seq_model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=30,
    verbose=1,
)

Epoch 1/30
80/80 [==============================] - 1162s 14s/step - loss: 0.6882 - binary_accuracy: 0.5783 - val_loss: 0.6833 - val_binary_accuracy: 0.5785
Epoch 2/30
68/80 [========================>.....] - ETA: 2:16 - loss: 0.6820 - binary_accuracy: 0.5779

KeyboardInterrupt: 